In [1]:
#import sys
#sys.path.append('/home/gonzalo/caffe/python') 

import sys
caffe_root = '../caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
import dlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tempfile
import os

In [4]:
from caffe.proto import caffe_pb2

def solver(train_net_path, test_net_path=None, base_lr=0.0001):
    s = caffe_pb2.SolverParameter()

    # Specify locations of the train and (maybe) test networks.
    s.train_net = train_net_path
    if test_net_path is not None:
        s.test_net.append(test_net_path)
        s.test_interval = 50  # Test after every 50 training iterations.
        s.test_iter.append(10) # Test on 10 batches each time we test.

    # The number of iterations over which to average the gradient.
    # Effectively boosts the training batch size by the given factor, without
    # affecting memory utilization.
    s.iter_size = 10
    
    s.max_iter = 1000     # # of times to update the net (training iterations)
    
    # Solve using the stochastic gradient descent (SGD) algorithm.
    # Other choices include 'Adam' and 'RMSProp'.
    s.type = 'SGD'

    # Set the initial learning rate for SGD.
    s.base_lr = base_lr

    # Set `lr_policy` to define how the learning rate changes during training.
    # Here, we 'step' the learning rate by multiplying it by a factor `gamma`
    # every `stepsize` iterations.
    s.lr_policy = 'step'
    s.gamma = 0.1
    s.stepsize = 50

    # Set other SGD hyperparameters. Setting a non-zero `momentum` takes a
    # weighted average of the current gradient and previous gradients to make
    # learning more stable. L2 weight decay regularizes learning, to help prevent
    # the model from overfitting.
    s.momentum = 0.9
    s.weight_decay = 5e-4

    # Display the current training loss and accuracy every 5 iterations.
    s.display = 5

    # Snapshots are files used to store networks trained.  Here, we'll
    # snapshot every 10K iterations -- ten times during training.
    s.snapshot = 150
    s.snapshot_prefix = '/home/gonzalo/models/fine_tune_VGG'#caffe_root + 'models/finetune_flickr_style/finetune_flickr_style'
    
    # Train on the GPU.  Using the CPU to train large networks is very slow.
    s.solver_mode = caffe_pb2.SolverParameter.GPU
    
    # Write the solver to a temporary file and return its filename.
    with tempfile.NamedTemporaryFile(delete=False) as f:
        f.write(str(s))
        return f.name

In [5]:
def run_solvers(niter, solvers, disp_interval=10):
    """Run solvers for niter iterations,
       returning the loss and accuracy recorded each iteration.
       `solvers` is a list of (name, solver) tuples."""
    blobs = ('loss', 'accuracy')
    loss, accuracy = ({name: np.zeros(niter) for name, _ in solvers}
                 for _ in blobs)
    for it in range(niter):
        for name, s in solvers:
            s.step(1)  # run a single SGD step in Caffe
            loss[name][it], accuracy[name][it] = (s.net.blobs[b].data.copy()
                                             for b in blobs)
        if it % disp_interval == 0 or it + 1 == niter:
            loss_disp = '; '.join('%s: loss=%.3f, accuracy=%2d%%' %
                                  (n, loss[n][it], np.round(100*accuracy[n][it]))
                                  for n, _ in solvers)
            print '%3d) %s' % (it, loss_disp)     
    # Save the learned weights from both nets.
    weight_dir = tempfile.mkdtemp()
    weights = {}
    for name, s in solvers:
        filename = 'weights.%s.caffemodel' % name
        weights[name] = os.path.join(weight_dir, filename)
        s.net.save(weights[name])
    return loss, accuracy, weights

In [ ]:
niter = 300  # number of iterations to train

# Reset style_solver as before.
proto_solver_filename = solver('/home/gonzalo/dev/vgg_face_caffe/VGG_FACE_train_val.prototxt')#primer_net(train=True))
proto_solver = caffe.get_solver(proto_solver_filename)
weights = '/home/gonzalo/models/fine_tune_VGG_iter_300.caffemodel'#'/home/gonzalo/dev/vgg_face_caffe/VGG_FACE.caffemodel'#
proto_solver.net.copy_from(weights)

print 'Running solver for %d iterations...' % niter
solvers = [('pretrained', proto_solver)]
loss, accuracy, weights = run_solvers(niter, solvers)
print 'Done.'

train_loss = loss['pretrained']
train_acc = accuracy['pretrained']
style_weights = weights['pretrained']

# Delete solvers to save memory.
del proto_solver, solvers

In [5]:
proto_solver.net.blobs.keys()

del proto_solver, solvers